In [1]:
%matplotlib widget
import numpy as np
from scipy import linalg, spatial, fft
from sklearn.experimental import enable_iterative_imputer
from sklearn import metrics, decomposition, impute
import matplotlib.pyplot as plt
# from read_WDF import read_WDF
import visualize as vis
import CR_search as cr
import preprocessing as pp
import calculate as cc
from read_WDF_class import WDF

# folder = "../../Raman/Data/Giuseppe/"
# file = "TFCD_ITOcell_532nm_p100_1s_carto_z20.wdf"
folder = "../../Raman/Data/Chloe/prob/"
file = "sableindien-x5-532streamline-map1.wdf"
folder = "/media/dejan/LJUBA/RamanSGR-Santiago/Santiago/"
file = "Na0_5_t_10-100um-70C_175°C_532nm-obj100_p100_10s_slice_Copy_Copy_Copy_Copy.wdf"
# file = "Na0_5_t_10-100um-70C_175°C_532nm-obj100_p100_10s_slice.wdf"
# file = "Na0_5_t_10-100um-70C_532nm-obj100_p100_10s_slice.wdf"
# folder = "../../Raman/Data/Maxime/CartosPlaques/M1C0/map_surface/"
# file = "M1C0_Map_Qontor_7x7cm_Surface1.wdf"
filename = folder + file

# rawspectra, x_values, params, map_params, origins = read_WDF(filename, verbose=True)
# nx, ny = list(map_params["NbSteps"][map_params["NbSteps"]>1])#[::-1]
fff = WDF(filename)



# Order:
# rawspectra, x_values = pp.order(rawspectra, x_values)

# vis.ShowSpectra(rawspectra, x_values);

In [2]:
vis.AllMaps(fff);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
vis.ShowSpectra(fff);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
xlabel, ylabel = fff.map_params["StepSizes"][fff.map_params["StepSizes"] > 0]

array([1.e+03, 5.e-01], dtype=float32)

In [2]:
vis.ShowSelected(fff);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
nx=10
ny=5
arr = np.arange(nx*ny)#.reshape(ny,nx)
arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [11]:
reordered = [arr[(xx // nx + 1)*nx - (xx % nx) -1] if (xx//nx) % 2 == 1 else arr[xx] for xx in range(nx*ny)]

In [12]:
np.array(reordered).reshape(ny,nx)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [19, 18, 17, 16, 15, 14, 13, 12, 11, 10],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [39, 38, 37, 36, 35, 34, 33, 32, 31, 30],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [17]:
reorder_inverted(arr, nx, ny)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [19, 18, 17, 16, 15, 14, 13, 12, 11, 10],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [39, 38, 37, 36, 35, 34, 33, 32, 31, 30],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [22]:
def reorder_inverted(arr, nx, ny):
    reordered = [arr[(xx // nx + 1)*nx - (xx % nx) -1] if (xx//nx) % 2 == 1 else arr[xx] for xx in range(nx*ny)]
    return np.array(reordered).reshape(ny, nx)
    
def reorder_inv(arr, nx, ny):
    arr = arr.reshape(ny, nx)
    return np.array([arr[i][::-1] if (i & 1) else arr[i] for i in range(ny)])

# or, if already in the right shape:

def reorder(arr):
    ny, nx = arr.shape
    return np.array([arr[i][::-1] if i&1 else arr[i] for i in range(ny)])

In [27]:
%timeit reorder_inverted(arr, nx, ny)

125 ms ± 8.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit reorder_inv(arr, nx, ny)

548 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
prd.reshape().ndim

1

In [31]:
prd = reorder_inv(arr, nx, ny)

In [32]:
reorder_inv(prd, nx, ny)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [29]:
np.all(reorder_inverted(arr, nx, ny) == reorder_inv(arr, nx, ny))

True

In [2]:
spectra = pp.correct_saturated(rawspectra, (ny, nx),
                               n_nearest_features=10, max_iter=64,
                               smoothen=True, lam=5, copy=True)

In [109]:
spectra[1853,444:452] = 50
spectra[2914, 880:885] = 55
spectra[1936, 193:199] = 65
spectra[2241, 862:866] = 65
spectra[2231, 177:182] = 65
spectra[2657, 203:208] = 65
spectra[1637, 290:295] = 80
spectra[918, 797:802] = 65
spectra[1510, 341:353] = 68

In [3]:
baseline = cc.baseline_als(spectra[:,120:], lam=80000)
# baseline -= np.min(baseline, axis=-1, keepdims=True)

spectra = spectra[:, 120:]

x_values = x_values[108:]

100%|██████████| 19881/19881 [01:26<00:00, 230.77it/s]


In [49]:
bl = vis.AllMaps(np.log(1+baseline.reshape(ny, nx, -1)), x_values);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
vis.ShowSpectra(spectra-baseline, sigma=x_values)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Cosmic Rays search 

In [52]:
# spectra -= cc.baseline_als(spectra, lam=200)
spectra -= np.min(spectra, axis=-1, keepdims=True)
spectra2 = (spectra - baseline) / \
            np.median(spectra - baseline, axis=-1, keepdims=True)
sisa = (np.max(spectra2, axis=0)-2*np.percentile(spectra2, 99, axis=0)
        +np.median(spectra2, axis=0))/np.median(spectra2, axis=0)
st_dev = np.std(sisa)

suspicious_positions = np.where(sisa > 2*st_dev)[0]
condition = spectra2.T[suspicious_positions] > 10*np.median(spectra2, axis=-1, keepdims=True)[suspicious_positions]
uslov = np.nonzero(condition)
suspects = np.unique([uslov[1][x] for x, y in itertools.groupby(uslov[0]) if len(list(y))<=np.ceil(len(uslov[0])/100)])
zez = vis.ShowSpectra(spectra2[suspects]);

IndexError: arrays used as indices must be of integer (or boolean) type

In [53]:
uslov

(array([], dtype=int64), array([], dtype=int64))

In [54]:
np.where(sisa > 2*st_dev)

(array([152, 153, 328, 416, 417, 418, 419, 420, 422]),)

In [31]:
len(suspicious_positions), condition.shape, spectra.shape, sisa.shape

(27, (27, 4347), (4347, 1015), (1015,))

In [34]:
np.where(sisa > 2*st_dev)

(array([ 33, 129, 169, 179, 180, 195, 196, 197, 198, 205, 206, 291, 292,
        293, 448, 582, 583, 799, 800, 863, 864, 865, 881, 882, 883, 884,
        922]),)

In [28]:
uslov = np.nonzero(condition)

In [32]:
uslov

(array([ 0,  1,  2,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  5,  6,  7,  8,  9,  9, 10, 10, 11, 11, 12, 13, 14, 15, 16, 17,
        18, 19]),
 array([  43,  494,  494, 1310,   37,  159,  479,  519,  600,  640,  681,
         759,  870,  896, 1101, 1103, 1141, 1143,  965,  965,  809,   44,
          44,  912,   44,  912,   44,  912,  119,    4,  608,  608, 1262,
         495,  329, 1146]))

In [30]:
np.ce

array([  43.,  494.,  494., 1310.,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
        965.,  965.,  809.,   44.,   44.,  912.,   44.,  912.,   44.,
        912.,  119.,    4.,  608.,  608., 1262.,  495.,  329., 1146.])

In [27]:
np.delete(suspicious_positions, 4)

array([ 12,  70,  71, 136, 314, 315, 348, 365, 366, 367, 368, 383, 389,
       590, 591, 630, 742, 745, 784])

In [38]:
[uslov[1][x] for x, y in itertools.groupby(uslov[0]) if len(list(y))<=np.ceil(len(uslov[0])/100)]

[43,
 494,
 494,
 1310,
 159,
 479,
 519,
 600,
 870,
 896,
 1101,
 1103,
 1141,
 1143,
 965,
 965]

In [31]:
import itertools
np.where(uslov[0] != 4, uslov[1], np.nan)

In [55]:
vis.ShowSpectra(spectra2.T[suspicious_positions])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
vis.ShowSpectra(spectra.T[suspicious_positions])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Saturated & Zero spectra

In [4]:
plt.close("all")
trt = pp.find_saturated(rawspectra)
almostraw = pp.correct_zeros(rawspectra, copy=True)
sat_idx = np.where(almostraw==0)
almostraw[sat_idx] = np.nan
novica = np.zeros_like(almostraw)
novica[sat_idx] = spectra[sat_idx]
novica[novica==0] = np.nan
ss = np.stack((rawspectra[trt]-10000, spectra[trt], novica[trt]), axis=-1)
vis.ShowSpectra(ss, x_values, title=["Spectra N°"+str(x) for x in trt]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# =============================================================================
#                                     PCA...
# =============================================================================
npca = 50
pca = decomposition.PCA(n_components=npca)
pca_fit = pca.fit(spectra)

spectra_reduced = pca_fit.transform(spectra)
spectra_denoised = pca_fit.inverse_transform(spectra_reduced)

guz = pca.get_covariance()

In [4]:
vis.AllMaps(spectra_reduced.reshape(ny, nx, -1), components=pca.components_, components_sigma=x_values)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.figure()
plt.plot(guz[:,674])
# plt.plot(guz[:,179])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
plt.close('all')
plt.imshow(np.log(guz))
# plt.imshow(np.log(guz))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.close('all')
plt.figure()
plt.plot(spectra[674])
plt.plot(rawspectra[674]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.figure()
plt.plot(np.log(pca.explained_variance_ratio_), 'bo')
plt.plot(np.log(pca.explained_variance_), 'xg')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# =============================================================================
#                  showing the smoothed spectra
# =============================================================================
valid_idx = np.ones(npca, dtype=bool)
valid_idx[1] = 0
novica = pca.mean_ + np.dot(spectra_reduced[:, valid_idx], pca.components_[valid_idx])

In [ ]:
vis.AllMaps(novica.reshape(ny, nx, -1), sigma=x_values)

In [ ]:
vis.ShowSpectra(novica, sigma=x_values)

```Python
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from time import time

estimators = [
    BayesianRidge(),
    DecisionTreeRegressor(max_features='sqrt'),
    ExtraTreesRegressor(n_estimators=10),
    KNeighborsRegressor(n_neighbors=8)
]


zero_idx = pp.find_zeros(rawspectra)
rawspectra[zero_idx] = np.median(rawspectra, axis=0)

sat = pp.find_saturated(rawspectra)

saturated_idx = np.where(rawspectra==0)

# rawspectra[saturated_idx] = np.nan
rawspectra[saturated_idx] = np.nan

trt = pp.get_neighbourhood(sat, map_shape)
# create an array so that trt[vrackalica] = sat
vrackalica = np.array([np.argwhere(trt==i)[0][0] for i in sat])

krk = dict()
tajming = dict()
for jj, est in enumerate(estimators):
    imp = impute.IterativeImputer(n_nearest_features=8, max_iter=44, estimator=est)
    start = time()
    popravljeni = imp.fit_transform(rawspectra[trt].T).T[vrackalica]
    tajming[str(est)] = time()-start
    spectra = np.copy(rawspectra)
    spectra[sat] = popravljeni
    ss = np.stack((rawspectra[sat]-1000, spectra[sat]), axis=-1)
    krk[jj] = vis.ShowSpectra(ss, x_values, title=[str(x) for x in sat])
    krk[jj].fig.suptitle(str(est))
```

In [ ]:
# if smooth:
#     upeglani = cc.baseline_als(popravljeni, lam=len(x_values)//5, p=0.6)

#     is_changed = np.diff(saturated_idx[0], prepend=sat[0])!=0
#     renormalizovani = []
#     i = 0
#     for cond in is_changed:
#         if cond:
#             i+=1
#         renormalizovani.append(i)

#     novi = np.copy(saturated_idx)
#     novi[0] = np.array(renormalizovani)
#     novi = tuple(novi)


#     spectra[saturated_idx] = upeglani[novi]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
np.meshgrid(np.arange(ny), np.arange(nx), indexing='ij')

[array([[ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 2,  2,  2, ...,  2,  2,  2],
        ...,
        [60, 60, 60, ..., 60, 60, 60],
        [61, 61, 61, ..., 61, 61, 61],
        [62, 62, 62, ..., 62, 62, 62]]),
 array([[ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68],
        ...,
        [ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68]])]

In [139]:
plt.figure()
plt.plot(spectra[595])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …